## 3.5 출력층 설계하기

- 머신러닝 문제는 분류와 회귀로 나뉜다.
- 신경망은 분류와 회귀 모두 사용 가능하다.
- 일반적으로 분류에는 소프트맥스 함수를, 회귀에는 항등 함수를 사용한다.

&nbsp;

- 분류: 카테고리가 알고 싶을 때 (사진 속 인물 성별 분류, ...)
- 회귀: 숫자를 예측하고 싶을 때 (사진 속 인물의 몸무게 예측, ...)

## 3.5.1 항등 함수와 소프트맥스 함수 구현하기

- 항등 함수(identity function)는 입력을 그대로 출력한다.
    ```mermaid
    graph LR
        a1((a1)) --> sigma1[σ] --> y1((y1))
        a2((a2)) --> sigma2[σ] --> y2((y2))
        a3((a3)) --> sigma3[σ] --> y3((y3))
    ```
- 소프트맥스 함수(softmax function)은 조금 더 복잡하다.
    - n을 출력층의 전체 뉴런 수(카테고리 수)라고 할 때, k번째 입력 신호 $a_k$에 대한 소프트맥스 함수 $y_k$는 다음과 같다.
        $$
        y_k = \frac{e^{a_k}}{\sum_{i=1}^{n} e^{a_i}}
        $$
    - 현재 입력 신호에 대한 지수함수를 전체 입력 신호에 대해 지수함수를 취한 합으로 나눈다.
    - 이에 따라 최종적으로 출력되는 신호는 모든 입력 신호의 영향을 받게 된다.
        ```mermaid
        graph LR
            %% 입력 노드
            a1((a1))
            a2((a2))
            a3((a3))

            %% 출력 노드
            y1((y1))
            y2((y2))
            y3((y3))

            %% softmax 작동 표현: 모든 입력 → 모든 출력
            a1 --> y1
            a2 --> y1
            a3 --> y1

            a1 --> y2
            a2 --> y2
            a3 --> y2

            a1 --> y3
            a2 --> y3
            a3 --> y3

            %% 연결 없는 σ 텍스트 노드
            sigma["σ (softmax)"]
        ```

In [10]:
# 소프트맥스 함수 구현
import numpy as np

def softmax_unsafe(a: np.array):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

a = np.array([0.3, 2.9, 4.0])
y = softmax_unsafe(a)
print(y)

# 이 구현은 논리적으로 문제가 없지만, 컴퓨터에서 수행하기에는 오버플로우에 취약하다는 문제가 있다.
testInput = np.array([1000, 2000, 3000])
testOutput = softmax_unsafe(testInput)
print(testOutput)

[0.01821127 0.24519181 0.73659691]
[nan nan nan]


/var/folders/bf/7t7t_z3n14n_qd_jqzpnt5j40000gp/T/ipykernel_96273/2051236858.py:5: RuntimeWarning: overflow encountered in exp
  exp_a = np.exp(a)
/var/folders/bf/7t7t_z3n14n_qd_jqzpnt5j40000gp/T/ipykernel_96273/2051236858.py:7: RuntimeWarning: invalid value encountered in divide
  y = exp_a / sum_exp_a


- 위에서 보듯이 소프트맥스 함수를 그대로 코드로 옮겨서 계산하는 경우 지수함수 특성 상 오버플로우 가능성이 있다. 분자와 분모가 너무 커지기 때문이다.
- 다만 아래와 같이 지수 함수에 대입하기 전에 어떤 정수를 빼도 결과에는 영향을 미치지 않는다는 성질을 이용하면 오버플로우 위험성을 낮출 수 있다.
    ![](./imgs/3-3.jpg)
- 대부분 입력 신호 중 가장 큰 수를 C로 정해 뺀다.

In [ ]:
def softmax(a: np.array):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

testInput = np.array([0, -10, -20])
testOutput1 = softmax_unsafe(testInput)
testOutput2 = softmax(testInput)
print(testOutput1)
print(testOutput2)
print(testOutput1 == testOutput2)

# **소프트맥스 함수 출력값의 합은 1이다**
print(np.sum(testOutput1))

[9.99954600e-01 4.53978686e-05 2.06106005e-09]
[9.99954600e-01 4.53978686e-05 2.06106005e-09]
[ True  True  True]
1.0


### 3.5.3 소프트맥스 함수의 특징
- 위에서 봤듯이 소프트맥스 함수의 출력은 0에서 1.0 사이의 실수이며, 총합은 1이다.
- 즉, 출력값을 확률로 이해할 수 있다.
- 분류 문제를 푼다면, 소프트맥스 함수의 출력 신호의 수를 분류 클래스 수에 맞게 두고, 각각을 입력 신호가 해당 클래스에 해당할 확률로 볼 수 있다는 것이다.

&nbsp;

- 주의: 소프트맥스 함수는 단조 증가 함수(정의역 원소 a, b가 a<=b인 경우, f(a) <= f(b)를 항상 만족하는 함수)이므로 직전 입력 신호의 대소 관계를 변하게 하지는 않는다.
- 즉, 분류 문제를 풀 때 마지막 출력층의 소프트맥스 함수는 생략해도 된다. 각 케이스에 대한 순위는 소프트맥스 함수를 거치지 않아도 그 시점에 이미 정해져 있다는 것이다.
- 지수 함수 계산은 고비용의 컴퓨팅 작업이다.
- 물론 학습 단계에서는 소프트맥스 함수 계산 결과가 필요하므로 계산한다. (영택: 각 클래스에 대해 정확한 컨피던스가 필요한 경우는?)

## 3.6 손글씨 숫자 인식

- 손글씨 숫자를 분류하는 실습을 진행한다.
- 학습은 이미 되었다고 치고, 제공되는 매개변수를 이용해 추론만 구현한다.
- 이렇게 신경망에 매개변수를 세팅해 추론하는 과정을 신경망의 순전파(forward propagation)라고도 한다.
    - 학습 -> 추론: 원래는 학습을 먼저 거쳐 가중치 매개변수를 세팅한다. 그 후 해당 매개변수를 이용해 추론한다.

### 3.6.1 MNIST 데이터셋

- 0부터 9까지의 손글씨 숫자 이미지 데이터
- 구성: 학습용 60,000장, 시험용 10,000장
- 포맷: 28 * 28 1채널 회색조 이미지로, 각 픽셀의 컬러 데이터는 0에서 255까지
- 이미지마다 무슨 숫자인지 레이블이 있음
- 데이터셋 다운로드
    ```
    cd ch3
    pwd # 경로 확인: .../DeepLearningFromScratch/ch3

    curl -OL https://github.com/WegraLee/deep-learning-from-scratch/raw/master/dataset/t10k-images-idx3-ubyte.gz
    curl -OL https://github.com/WegraLee/deep-learning-from-scratch/raw/master/dataset/t10k-labels-idx1-ubyte.gz
    curl -OL https://github.com/WegraLee/deep-learning-from-scratch/raw/master/dataset/train-images-idx3-ubyte.gz
    curl -OL https://github.com/WegraLee/deep-learning-from-scratch/raw/master/dataset/train-labels-idx1-ubyte.gz
    ```

In [19]:
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)

print(x_train.shape) # 훈련 이미지
print(t_train.shape) # 훈련 레이블
print(x_test.shape) # 시험 이미지
print(t_test.shape) # 시험 레이블

"""load_mnist 함수 사용법

Parameters
----------
normalize : 이미지의 픽셀 값을 0.0~1.0 사이의 값으로 정규화할지 정한다.
one_hot_label : 
    one_hot_label이 True면、레이블을 원-핫(one-hot) 배열로 돌려준다.
    one-hot 배열은 예를 들어 [0,0,1,0,0,0,0,0,0,0]처럼 한 원소만 1인 배열이다.
    false면 레이블 자체를 반환한다.
flatten : 입력 이미지를 1차원 배열로 만들지를 정한다. 

Returns
-------
(훈련 이미지, 훈련 레이블), (시험 이미지, 시험 레이블)
"""

(60000, 784)
(60000,)
(10000, 784)
(10000,)


'load_mnist 함수 사용법\n\nParameters\n----------\nnormalize : 이미지의 픽셀 값을 0.0~1.0 사이의 값으로 정규화할지 정한다.\none_hot_label : \n    one_hot_label이 True면、레이블을 원-핫(one-hot) 배열로 돌려준다.\n    one-hot 배열은 예를 들어 [0,0,1,0,0,0,0,0,0,0]처럼 한 원소만 1인 배열이다.\n    false면 레이블 자체를 반환한다.\nflatten : 입력 이미지를 1차원 배열로 만들지를 정한다. \n\nReturns\n-------\n(훈련 이미지, 훈련 레이블), (시험 이미지, 시험 레이블)\n'

In [ ]:
# 데이터셋 훑어보기
from PIL import Image

def img_show(img: np.array):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)

img = x_train[1000]
label = t_train[1000]
print(label)

print(img.shape) # (784, ) -> 한 줄로 나열되어 있음. load_mnist에 flatten=True를 넘겼기 때문에.
img = img.reshape(28, 28) # 다시 2차원 배열로
print(img.shape) # (28, 28) -> 이제 렌더링할 수 있다

img_show(img)

0
(784,)
(28, 28)


## 3.6.2 신경망 추론 처리
- 설계
    - 입력층 뉴런 784개 (이미지 크기 28 * 28)
    - 출력층 뉴런 10개 (클래스 0~9)
    - 은닉층은 두 층으로 하고, 첫 번째 은닉층에는 50개 뉴런을, 두 번째 은닉층에는 100개 뉴런을 배치. 50과 100은 임의로 정한 수
    

In [34]:
import os
import pickle

def sigmoid(x: np.array) -> np.array:
    return 1 / (1 + np.exp(-x))
    
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)
    return x_test, t_test

def init_network():
    # sample_weight.pkl: 이미 학습된 매개변수 객체를 직렬화해둔 것
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)

    return network

def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

In [49]:
# 정확도 테스트

import time

start = time.time()

x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i]) # ex> [0.1, 0.3, 0.2, ..., 0.04] -> 인덱스가 곧 클래스(레이블)이고 값은 각 클래스 별 확률을 나타냄
    p = np.argmax(y) # 가장 확률이 높은 인덱스(레이블)
    if p == t[i]: # 정답이면 +1
        accuracy_cnt += 1

print("Accuracy: " + str(float(accuracy_cnt) / len(x)))
print("elpased: " + str(time.time() - start))

Accuracy: 0.9207
elpased: 0.16651320457458496


/var/folders/bf/7t7t_z3n14n_qd_jqzpnt5j40000gp/T/ipykernel_96273/2561148362.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


- 정확도 92% 정도
- 뒤로 나가면서 높여볼 예정

&nbsp;

- 정규화(normalization)
    - load_mnist 패러미터 중 normalized=True로 지정
    - 0 ~ 255인 각 픽셀의 컬러값을 255로 나누어 0.0 ~ 1.0 범위로 변환함
- 전처리: 신경망의 입력 데이터에 특정 변환을
    - 신경망에 투입하기 전에 입력 데이터에 변환 작업을 수행
- 여기서는 전처리 작업의 일환으로 아주 간단하게 정규화를 수행한 것
- 현업에서는 데이터 분포를 고려해 전처리를 하는 경우가 많음.
    - ex> 데이터 전체 분포를 고려해 데이터가 0 중심으로 분포하도록 이동, 데이터 확산 범위 제한, ...
    - 그 외에 전체 데이터를 균일하게 분포시키는 것을 데이터 백색화라고 함


### 3.6.3 배치 처리

- 가중치의 shape을 다시 한 번 보자.

In [41]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network["W1"], network["W2"], network["W3"]

print("x.shape")
print(x.shape)
print("x[0].shape")
print(x[0].shape)
print("W1.shape")
print(W1.shape)
print("W2.shape")
print(W2.shape)
print("W3.shape")
print(W3.shape)

x.shape
(10000, 784)
x[0].shape
(784,)
W1.shape
(784, 50)
W2.shape
(50, 100)
W3.shape
(100, 10)


- 위의 코드에서 그랬듯 이미지 한 장 씩만 넣게 되면 아래와 같은 흐름으로 784개의 픽셀이 10개의 레이블 별 확률로 처리된다.

    | 레이어 | 형상 |
    |-----|-----|
    | X | **784** |
    | W1 | **784** * *_50_* |
    | W2 | *_50_* * **100** |
    | W3 | **100** * *_10_* |
    | Y | *_10_* |

- 그런데 배치로 여러 장을 한 번에 넘길 수도 있다. 예를 들어 배치 사이즈(한 번에 처리할 개수)를 100이라고 하자.

    | 레이어 | 형상 |
    |-----|-----|
    | X | **100** * 784 |
    | W1 | 784 * 50 |
    | W2 | 50 * 100 |
    | W3 | 100 * 10 |
    | Y | **100** * 10 |

    - 마지막에 최종 100장의 결과가 함께 반환된다.
    - 이것이 바로 배치 처리다. 배치는 묶음이다.
    - [NOTE] 배치 처리가 컴퓨팅 부담을 오히려 줄일 수도 있다. 수치계산 라이브러리들은 큰 배열에 최적화되어 있고, I/O 버스로 데이터가 이동하는 빈도를 줄인다.

In [50]:
# 배치 추론 구현

import time

start = time.time()

x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0
for i in range(0, len(x), batch_size): # i는 0, 100, 200, ...으로 커진다
    x_batch = x[i:i+batch_size] # 100개씩 데이터를 자른다 0..<100, 100..<200, ...
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) # 1번째 차원을 축으로 해서 최대값의 인덱스를 찾는다. 즉, 단건 각각에서의 최대값의 인덱스이다.
    accuracy_cnt += np.sum(p == t[i:i+batch_size]) # 일괄적으로 레이블과 비교한다.

print("Accuracy: " + str(float(accuracy_cnt) / len(x)))
print("elpased: " + str(time.time() - start))

Accuracy: 0.9207
elpased: 0.050425052642822266


/var/folders/bf/7t7t_z3n14n_qd_jqzpnt5j40000gp/T/ipykernel_96273/2561148362.py:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


## 3.7 정리

- 신경망은 각 층의 뉴런이 다른 측의 뉴런으로 신호를 전달한다는 점에서 퍼셉트론과 비슷한 부분이 있다.
- 다만 다음 뉴런으로 갈 때 신호 활성화 여부를 결정하는 활성화 함수가 다르다. 신경망에서는 단순 계단 함수가 아니라 시그모이드 함수를 쓴다.